In [1]:
## the HuggingFace libraries are not already installed in Colab
!pip install transformers==4.17 # this will also install tokenizers 
!pip install datasets
!pip install optuna
!pip install ray

     |████████████████████████████████| 3.8 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 64.1 MB/s 
     |████████████████████████████████| 77 kB 8.8 MB/s 
     |████████████████████████████████| 596 kB 84.6 MB/s 
     |████████████████████████████████| 6.6 MB 56.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 8.7 MB/s 
     |████████████████████████████████| 136 kB 50.9 MB/s 
     |████████████████████████████████| 1.1 MB 63.6 MB/s 
     |████████████████████████████████| 212 kB 51.6 MB/s 
     |████████████████████████████████| 127 kB 76.8 MB/s 
     |████████████████████████████████| 144 kB 73.1 MB/s 
     |████████████████████████████████| 94 kB 4.8 MB/s 
     |████████████████████████████████| 271 kB 94.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [2]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import numpy as np 
import matplotlib.pyplot as plt
import nltk
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, roc_curve, auc
from nltk.tokenize import sent_tokenize
import pandas as pd
import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer, RobertaForSequenceClassification,RobertaModel, Trainer, TrainingArguments
from transformers import AdamW, get_linear_schedule_with_warmup
import datasets
from ray.tune.schedulers import PopulationBasedTraining,ASHAScheduler
from ray import tune
from ray.tune import uniform, CLIReporter
import optuna
from random import randint
from sklearn.metrics import accuracy_score
from ray.tune.stopper import TrialPlateauStopper
import os

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


Load Data

In [3]:
data = datasets.load_dataset("tweet_eval", "emotion")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
### number of labels
num_labels = len(np.unique(data['train']['label']))
print (num_labels)

4


Hyperparameters for model

In [5]:
# Hyper-parameters
pretrained_model_name = "roberta-base" # we can change this, and play with any model from HuggingFace! 
# BERT-base: "bert-base-cased" #Sensitive to lower and upper case
# BERT-large: "bert-large-cased"
# RoBERTa-base: "roberta-base"
# RoBERTa-large: "roberta-large"
# etc ... 
max_length = 50
batch_size = 32
d_in = 768 # bert-base / roberta-base
d_h = 512 # 512 
d_out = num_labels # 2 classes so 1 is enough
freeze_pretrained = False

Tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name,
    cache_dir = "."
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Hyperparameters

In [7]:
# Hyper-parameters
n_epochs = 10
lr = 3e-5
warmup = 0.06
max_gradient_norm = 10
patience = 3

Fine-tune model class

In [8]:
class FinetuneModel(nn.Module):
    """Pretrained Model + classifier
    """
    def __init__(self, pretrained_model, d_in, d_h, d_out, dropout, freeze_pretrained=False):
        super(FinetuneModel, self).__init__()

        # Instantiate BERT model
        self.pretrained_model = pretrained_model

        # Instantiate a 2-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_in, d_h),
            nn.ReLU(),
            nn.Dropout(float(dropout)),
            nn.Linear(d_h, d_out)
        )

        # Freeze the encoder model
        if freeze_pretrained:
            for param in self.pretrained_model.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        # Feed input to pretrained model
        pre_outputs = self.pretrained_model(input_ids = input_ids,attention_mask = attention_mask)
        # print (pre_outputs[0].shape,pre_outputs[1].shape)
        # print (pre_outputs.keys())
        hidden_state = pre_outputs[0]
        hidden_state = hidden_state[:,0,:]
        logits = self.classifier(hidden_state)
        # print(outputs.keys()) # ["last_hidden_state", "pooler_output", "hidden_states", "attention"] last hidden state 才是最后的hidden
        # print (outputs[0].shape)
        # Extract the last hidden state
        # last_hidden = outputs["last_hidden_state"] # shape: (batch_size, sequence_length, d_in)
        
        # # Use the `[CLS]` token of the last hidden state for classification task
        # last_hidden_state_cls = last_hidden[:, 0, :] # shape: (batch_size, d_in)

        # Feed input to classifier to compute logits
        # logits = self.classifier(outputs[0]) # shape: (batch_size, 1)

        return logits

Dataset class

In [9]:
class EmotionsDataset():

    def __init__(self, tokenizer, df,max_length):
        super(EmotionsDataset, self).__init__()

        self.tokenizer = tokenizer
        self.max_seq_len = max_length
        self.input_ids, self.attention_mask,self.token_type_ids,self.label = self.get_input(df)
        

    def __len__(self):
        return len(self.label)
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens_seq) # convert to id
        input_ids += padding   # add padding
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len) # for identifying next sentence prediction
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids 

    def get_input(self,df):          
        tweet = df['text'].values
        label = df['label'].values
        tokens_seq = list(map(self.tokenizer.tokenize,tweet))
        result = list(map(self.trunate_and_pad, tokens_seq))
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]

        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(label).type(torch.long)
               )

    def __getitem__(self, item):
        return self.input_ids[item], self.attention_mask[item],self.token_type_ids[item],self.label[item]

Function to load data

In [10]:
def load_data(data_dir = None):
    if load_data  == None:
        datasets.load_dataset('tweet_eval','emotion')
    data_train = datasets.Dataset.to_pandas(data["train"])
    data_val = datasets.Dataset.to_pandas(data["validation"])
    data_test = datasets.Dataset.to_pandas(data["test"])
    ## Emotion dataset
    train_dataset = EmotionsDataset(tokenizer, data_train, max_length)
    val_dataset = EmotionsDataset(tokenizer, data_val, max_length)
    test_dataset = EmotionsDataset(tokenizer, data_test, max_length)
    ## dataloader
    # train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
    # val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
    # test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)
    return train_dataset,val_dataset,test_dataset

Config 

In [11]:
### config_model
config_model = AutoConfig.from_pretrained(
    pretrained_model_name, 
    output_hidden_states = True,
    output_attention = False
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Train function

In [12]:
def set_seed(seed_value = 0):
    """Set seed for reproducibility.
    """
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(train_loader, model, optimizer, scheduler,loss_fct):
    model.train()
    train_losses = []
    correct_preds = 0
    pred_class = []
    actual_class = []
    tqdm_loader = tqdm(train_loader)
    for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in enumerate (tqdm_loader):
        model.zero_grad()
  

        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        segments = segments.to(device)
        labels = labels.to(device)

        logits = model(input_ids, attention_mask) # forward pass (batch_size, 1)
        loss = loss_fct(logits,labels)
        probs = nn.functional.softmax(logits,dim = -1)
        _,out_classes = torch.max(logits,dim = 1)
        ## for weighted accuracy
        pred_class.extend(out_classes.cpu().numpy())
        actual_class.extend(labels.cpu().numpy())

        correct = (out_classes == labels).sum()
        correct_preds += correct.item()
        train_losses.append(loss.item())

        loss.backward() # backward pass
        ## clip grad
        nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step() # weights update
        optimizer.zero_grad()
        
    train_loss = np.mean(train_losses)
    epoch_accuracy = correct_preds /len(train_loader.dataset)
    # epoch_accuracy = balanced_accuracy_score(actual_class,pred_class)

    return train_loss,epoch_accuracy

from sklearn.metrics import roc_auc_score, f1_score
def eval_multiclass(loader, model,loss_fct):
    model.eval()
    val_losses = []
    val_probs = []
    acc_correct = 0
    iter = 0
    log_interval = 100
    sigmoid = nn.Sigmoid()
    log_interval = 50
    count = 0
    val_labels = []
    val_targets = []
    pred_class = []
    actual_class = []
  
    # threshold = torch.tensor([0.5]).to(device)
    # loader = tqdm(loader)
    for (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in loader:
        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        segments = segments.to(device)
        # print (attention_mask.shape,labels.shape)
        # print (segments.shape)
        
        with torch.no_grad():
            valid_logits = model(input_ids, attention_mask) # raw logits
            # print (f'logits = {valid_logits}')
            # print(f'labels = {labels}')
            valid_loss = loss_fct(valid_logits,labels) # loss
            valid_probs = nn.functional.softmax(valid_logits,dim = -1)
            _,out_classes = torch.max(valid_logits,dim = 1)
            # if count % log_interval == 0:
            #   print (f'labels = {labels}')
            #   print (f'predicted labels = {out_classes}')
            f1_target = torch.argmax(valid_probs,dim= 1)
            # print (f1_target)
            correct = (out_classes == labels).sum()
            acc_correct += correct.item()
            # weighted accuracy

            pred_class.extend(out_classes.cpu().numpy())
            actual_class.extend(labels.cpu().numpy())


            # print (valid_probs.shape)
            val_probs.extend(valid_probs[:,:].cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_losses.append(valid_loss.item())
            val_targets.extend(f1_target.cpu().numpy())
            count +=1
            # print (count)
    val_loss = np.mean(val_losses)
    val_probs = np.array(val_probs)
    val_labels = np.array(val_labels)
    val_targets = np.array(val_targets)
    valid_accuracy = acc_correct/(len(loader.dataset))
    # valid_accuracy = balanced_accuracy_score(actual_class,pred_class)

    
    val_auc = 100* roc_auc_score(val_labels, val_probs, multi_class='ovr')
    val_f1_score = f1_score(val_labels,val_targets,average = 'weighted')

    return val_loss, val_auc, val_targets, val_labels,valid_accuracy,val_f1_score

In [13]:
def train_function(config, checkpoint_dir='fine-tuned_model', data_dir=None):

    # if checkpoint_dir == None:
    #   checkpoint_dir = 'fine-tuned_model'
    ## define model
    model = RobertaModel.from_pretrained("roberta-base",config = config_model)
    model = FinetuneModel(model,d_in,config['d_h'],d_out,config['dropout'],freeze_pretrained) #pretrained_model, d_in, d_h, d_out, freeze_pretrained=False

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
    model.to(device)
    print (f'is GPU available? {torch.cuda.is_available()}')
    print (f'device used: {device}')
    ### Optimizer
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
            {
                    'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                    'weight_decay':config['weight_decay']
            },
            {
                    'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                    'weight_decay':0.0
            }
    ]
    # optimizer = AdamW(optimizer_grouped_parameters, lr=config['lr'],betas = (config['beta1'],config['beta2']),eps = config['eps'])
    if config["optimizer"] == "adamW":
      optimizer = AdamW(optimizer_grouped_parameters, lr=config['lr'],betas = (config['beta1'],config['beta2']),eps = config['eps'])
    elif config["optimizer"] == "adam":
      optimizer = optim.Adam(model.parameters(), lr = config['lr'])
    elif config["optimizer"] == "SGD":
      optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config["momentum"])
      
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=config['factor'], patience=config['patience'])
    criterion = nn.CrossEntropyLoss()

    # if checkpoint_dir:
    #     model_state, optimizer_state = torch.load(
    #         os.path.join(checkpoint_dir, "checkpoint"))
    #     model.load_state_dict(model_state)
    #     optimizer.load_state_dict(optimizer_state)

    trainset, valset,_ = load_data(data_dir)


    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=int(config["batch_size"]),
        shuffle=True)
    valloader = torch.utils.data.DataLoader(
        valset,
        batch_size=int(config["batch_size"]),
        shuffle=False)
    ## Training ####
    for epoch in range(config['num_epochs']):  # loop over the dataset multiple times
        train_loss, epoch_accuracy = train(trainloader, model, optimizer, scheduler, criterion)

        # Validation loss
        val_loss, val_auc, val_targets, val_labels,valid_accuracy,val_f1_score = eval_multiclass(valloader, model, criterion)
                
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            print (f'checkpoint path : {path}')
            torch.save((model.state_dict(), optimizer.state_dict(),scheduler.state_dict()), path)
            print ('Model Saved!')
        tune.report(loss=val_loss, accuracy=valid_accuracy)
        ## scheduler
        scheduler.step(valid_accuracy)
    print("Finished Training")

Test accuracy function for test set

In [14]:
def test_accuracy(model, device="cpu"):
    _,_,testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False)
    criterion = nn.CrossEntropyLoss()
    correct = 0
    total = 0
    test_loss = 0.
    test_steps = 0
    with torch.no_grad():
        for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in enumerate (testloader):
            batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels = batch_seqs.to(device), batch_seq_masks.to(device), batch_seq_segments.to(device), batch_labels.to(device)
            outputs = model(batch_seqs, batch_seq_masks)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs,batch_labels)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()
            test_loss += loss.cpu().numpy()
            test_steps += 1

    return correct / total, test_loss/test_steps

In [15]:
config = {
    "num_epochs": 10,
    "lr": 1e-5,
    "batch_size": 16,
    "weight_decay": 0.05,
    "beta1" : 0.9,
    "beta2" : 0.995,
    "eps": 1e-8,
    "max_gradient_norm": 8,
    "d_h" : 512,
    "dropout": 0.1,
    "factor": 0.5,
    'patience': 2,
    "optimizer": tune.choice(["adam", "SGD", "adamW"]),
    "momentum": 0.9
}

In [16]:
def main(num_samples=10,max_num_epochs =10, gpus_per_trial=1):
    load_data()
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    # ASHA scheduler
    scheduler = ASHAScheduler(
        metric="accuracy",
        mode="max",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    result = tune.run(
        train_function,
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter = reporter,
        )

    best_trial = result.get_best_trial("accuracy", "max", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    # best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    # device = "cpu"
    model = RobertaModel.from_pretrained("roberta-base",config = config_model)
    best_trained_model = FinetuneModel(model,d_in,best_trial.config['d_h'],d_out,best_trial.config['dropout'],freeze_pretrained)
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state,scheduler_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc,test_loss = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {} , test loss : {}".format(test_acc,test_loss))
    print (f'scheduler : {scheduler_state.keys()}')



Tuning on original dataset

In [17]:
main(num_samples=10,max_num_epochs=10, gpus_per_trial=1)

2022-04-14 07:49:46,980	INFO logger.py:606 -- pip install "ray[tune]" to see TensorBoard files.
2022-04-14 07:49:46,982	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2022-04-14 07:49:48 (running for 00:00:01.56)
Memory usage on this node: 2.2/25.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/4 CPUs, 1.0/1 GPUs, 0.0/14.34 GiB heap, 0.0/7.17 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/train_function_2022-04-14_07-49-46
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+----------------------------+----------+----------------+-------------+
| Trial name                 | status   | loc            | optimizer   |
|----------------------------+----------+----------------+-------------|
| train_function_748af_00000 | RUNNING  | 172.28.0.2:322 | adamW       |
| train_function_748af_00001 | PENDING  |                | adamW       |
| train_function_748af_00002 | PENDING  |                | adam        |
| train_function_748af_00003 | PENDING  |                | SGD         |
| train_function_748af_0

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]
Downloading:   1%|          | 2.94M/478M [00:00<00:16, 30.8MB/s]
Downloading:   1%|▏         | 6.45M/478M [00:00<00:14, 34.3MB/s]
Downloading:   2%|▏         | 10.0M/478M [00:00<00:13, 35.9MB/s]
Downloading:   3%|▎         | 13.9M/478M [00:00<00:12, 37.7MB/s]
Downloading:   4%|▍         | 18.2M/478M [00:00<00:11, 40.5MB/s]
Downloading:   5%|▍         | 22.4M/478M [00:00<00:11, 41.7MB/s]
Downloading:   6%|▌         | 26.8M/478M [00:00<00:11, 42.9MB/s]
Downloading:   6%|▋         | 31.0M/478M [00:00<00:10, 43.3MB/s]
Downloading:   7%|▋         | 35.2M/478M [00:00<00:10, 43.6MB/s]
Downloading:   8%|▊         | 39.4M/478M [00:01<00:10, 43.1MB/s]
Downloading:   9%|▉         | 43.7M/478M [00:01<00:10, 43.8MB/s]
Downloading:  10%|█         | 47.9M/478M [00:01<00:10, 43.9MB/s]
Downloading:  11%|█         | 52.1M/478M [00:01<00:10, 43.7MB/s]
Downloading:  12%|█▏        | 56.3M/478M [00:01<00:10, 43.8MB/s]
Downloading:  13%|█▎        | 60.6

== Status ==
Current time: 2022-04-14 07:49:59 (running for 00:00:12.34)
Memory usage on this node: 2.7/25.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/4 CPUs, 1.0/1 GPUs, 0.0/14.34 GiB heap, 0.0/7.17 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/train_function_2022-04-14_07-49-46
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+----------------------------+----------+----------------+-------------+
| Trial name                 | status   | loc            | optimizer   |
|----------------------------+----------+----------------+-------------|
| train_function_748af_00000 | RUNNING  | 172.28.0.2:322 | adamW       |
| train_function_748af_00001 | PENDING  |                | adamW       |
| train_function_748af_00002 | PENDING  |                | adam        |
| train_function_748af_00003 | PENDING  |                | SGD         |
| train_function_748af_0

Downloading:  46%|████▌     | 219M/478M [00:05<00:05, 47.6MB/s]
Downloading:  47%|████▋     | 224M/478M [00:05<00:05, 47.6MB/s]
Downloading:  48%|████▊     | 228M/478M [00:05<00:05, 46.0MB/s]
Downloading:  49%|████▊     | 233M/478M [00:05<00:05, 45.0MB/s]
Downloading:  50%|████▉     | 237M/478M [00:05<00:05, 45.8MB/s]
Downloading:  51%|█████     | 242M/478M [00:05<00:05, 46.1MB/s]
Downloading:  52%|█████▏    | 246M/478M [00:05<00:05, 46.6MB/s]
Downloading:  52%|█████▏    | 251M/478M [00:06<00:05, 46.7MB/s]
Downloading:  53%|█████▎    | 255M/478M [00:06<00:04, 46.9MB/s]
Downloading:  54%|█████▍    | 260M/478M [00:06<00:04, 46.4MB/s]
Downloading:  55%|█████▌    | 264M/478M [00:06<00:04, 46.1MB/s]
Downloading:  56%|█████▌    | 269M/478M [00:06<00:04, 46.1MB/s]
Downloading:  57%|█████▋    | 273M/478M [00:06<00:04, 46.2MB/s]
Downloading:  58%|█████▊    | 277M/478M [00:06<00:04, 46.1MB/s]
Downloading:  59%|█████▉    | 282M/478M [00:06<00:04, 46.9MB/s]
Downloading:  60%|█████▉    | 287M/478M 

...

(train_function pid=1430)   0%|          | 0/204 [00:00<?, ?it/s]
== Status ==
Current time: 2022-04-14 08:13:13 (running for 00:23:26.64)
Memory usage on this node: 4.0/25.5 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: 0.7727272727272727 | Iter 4.000: 0.7834224598930482 | Iter 2.000: 0.7727272727272727 | Iter 1.000: 0.7540106951871658
Resources requested: 2.0/4 CPUs, 1.0/1 GPUs, 0.0/14.34 GiB heap, 0.0/7.17 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/train_function_2022-04-14_07-49-46
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+----------------------------+------------+-----------------+-------------+----------+------------+----------------------+
| Trial name                 | status     | loc             | optimizer   |     loss |   accuracy |   training_iteration |
|----------------------------+------------+-----------------+-------------+----------+------------+----------------------|
| train_function_748af_00009 | RUNNING   

2022-04-14 08:14:10,371	INFO tune.py:639 -- Total run time: 1463.53 seconds (1463.09 seconds for the tuning loop).


Best trial config: {'num_epochs': 10, 'lr': 1e-05, 'batch_size': 16, 'weight_decay': 0.05, 'beta1': 0.9, 'beta2': 0.995, 'eps': 1e-08, 'max_gradient_norm': 8, 'd_h': 512, 'dropout': 0.1, 'factor': 0.5, 'patience': 2, 'optimizer': 'adam', 'momentum': 0.9}
Best trial final validation loss: 1.0733110674967368
Best trial final validation accuracy: 0.786096256684492


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Best trial test set accuracy: 0.8114004222378607 , test loss : 0.9032420500912808
scheduler : dict_keys(['factor', 'min_lrs', 'patience', 'verbose', 'cooldown', 'cooldown_counter', 'mode', 'threshold', 'threshold_mode', 'best', 'num_bad_epochs', 'mode_worse', 'eps', 'last_epoch', '_last_lr'])


Tuning on cleaned **dataset**

In [17]:
from nltk.stem import WordNetLemmatizer 
!pip install emoji
import emoji
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
!pip install emot
import re
from emot.emo_unicode import UNICODE_EMOJI

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

     |████████████████████████████████| 175 kB 8.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=a655ea3390033bff55e00ad4c7080faf11876783ebcba23f1a4db3599df4c687
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 61 kB 20 kB/s 


In [18]:
class CleanedEmotionsDataset():

    def __init__(self, tokenizer, df,max_length):
        super(CleanedEmotionsDataset, self).__init__()

        self.tokenizer = tokenizer
        self.max_seq_len = max_length
        self.input_ids, self.attention_mask,self.token_type_ids,self.label = self.get_input(df)
        

    def __len__(self):
        return len(self.label)
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens_seq) # convert to id
        input_ids += padding   # add padding
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len) # for identifying next sentence prediction
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids 

    # strip  @ with words and numbers
    def remove_pattern(self,input_txt):
        r = re.findall("@[\w]*", input_txt)
        for i in r:
            input_txt = re.sub(i, '', input_txt)
        input_txt = re.sub(r"\d+", "", input_txt)
        return input_txt  

    ## strip all @ and # with words following
    def strip_links(self,text):
        link_regex = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
        links = re.findall(link_regex, text)
        for link in links:
            text = text.replace(link[0], ', ')    
        return text
        
    def strip_all_entities(self,text):
        entity_prefixes = ['@','#']
        for separator in  string.punctuation:
            if separator not in entity_prefixes :
                text = text.replace(separator,' ')
        words = []
        for word in text.split():
            word = word.strip()
            if word:
                if word[0] not in entity_prefixes:
                    words.append(word)
        return ' '.join(words)

        # removing stopwords
    def removing_stopwords(self,text):
        text = repr(text)
        No_StopWords = [word for word in word_tokenize(text) if word.lower() not in stoplist ]
        # Convert list of tokens_without_stopwords to String type.
        words_string = ' '.join(No_StopWords)    
        return words_string

    ## lemmatization
    def lemmatization(self,text):
        # Converting words to their root forms
        lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
        return ' '.join(lemma)
    
    ## convert emoji to text
    def convert_emojis(self,text):
        for emot in UNICODE_EMOJI:
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
            text = text.replace('_', " ")
        return text

    def get_input(self,df):
        # df_tweet = self.clean_dataset(df['text'])
        ## clean off tags

        for i in range(len(df['text'])):
            df['text'][i] = df['text'][i].lower()
            df['text'][i] = self.remove_pattern(df['text'][i])
            df['text'][i] = self.convert_emojis(df['text'][i])
            # df['text'][i] = self.lemmatization(df['text'][i])
            # df['text'][i] = self.removing_stopwords(df['text'][i])
            # df['text'][i] = self.spell(self.lemmatization(df['text'][i]))
            
        tweet = df['text'].values
        # print (tweet)
        label = df['label'].values
        tokens_seq = list(map(self.tokenizer.tokenize,tweet))
        result = list(map(self.trunate_and_pad, tokens_seq))
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]

        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(label).type(torch.long)
               )

    def __getitem__(self, item):
        return self.input_ids[item], self.attention_mask[item],self.token_type_ids[item],self.label[item]

In [19]:
def load_data(data_dir = None):
    if load_data  == None:
        datasets.load_dataset('tweet_eval','emotion')
    data_train = datasets.Dataset.to_pandas(data["train"])
    data_val = datasets.Dataset.to_pandas(data["validation"])
    data_test = datasets.Dataset.to_pandas(data["test"])
    ## Emotion dataset
    train_dataset = CleanedEmotionsDataset(tokenizer, data_train, max_length)
    val_dataset = CleanedEmotionsDataset(tokenizer, data_val, max_length)
    test_dataset = CleanedEmotionsDataset(tokenizer, data_test, max_length)
    ## dataloader
    # train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
    # val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
    # test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)
    return train_dataset,val_dataset,test_dataset

In [20]:
main(num_samples=10, max_num_epochs=10, gpus_per_trial=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
2022-04-14 08:52:27,108	INFO logger.py:606 -- pip install "ray[tune]" to see TensorBoard files.
2022-04-14 08:52

== Status ==
Current time: 2022-04-14 08:52:28 (running for 00:00:00.99)
Memory usage on this node: 2.2/25.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/4 CPUs, 1.0/1 GPUs, 0.0/14.28 GiB heap, 0.0/7.14 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/train_function_2022-04-14_08-52-27
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+----------------------------+----------+----------------+-------------+
| Trial name                 | status   | loc            | optimizer   |
|----------------------------+----------+----------------+-------------|
| train_function_35c0f_00000 | RUNNING  | 172.28.0.2:374 | adam        |
| train_function_35c0f_00001 | PENDING  |                | adamW       |
| train_function_35c0f_00002 | PENDING  |                | SGD         |
| train_function_35c0f_00003 | PENDING  |                | adamW       |
| train_function_35c0f_0

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]
Downloading:   1%|          | 4.50M/478M [00:00<00:10, 47.1MB/s]
Downloading:   2%|▏         | 10.0M/478M [00:00<00:09, 53.5MB/s]
Downloading:   3%|▎         | 16.2M/478M [00:00<00:08, 58.7MB/s]
Downloading:   5%|▍         | 22.9M/478M [00:00<00:07, 63.4MB/s]
Downloading:   6%|▌         | 29.7M/478M [00:00<00:07, 66.2MB/s]
Downloading:   8%|▊         | 36.5M/478M [00:00<00:06, 67.8MB/s]
Downloading:   9%|▉         | 43.1M/478M [00:00<00:06, 68.2MB/s]
Downloading:  10%|█         | 49.6M/478M [00:00<00:06, 68.4MB/s]
Downloading:  12%|█▏        | 56.3M/478M [00:00<00:06, 68.8MB/s]
Downloading:  13%|█▎        | 63.2M/478M [00:01<00:06, 69.8MB/s]


== Status ==
Current time: 2022-04-14 08:52:33 (running for 00:00:06.13)
Memory usage on this node: 2.7/25.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/4 CPUs, 1.0/1 GPUs, 0.0/14.28 GiB heap, 0.0/7.14 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/train_function_2022-04-14_08-52-27
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+----------------------------+----------+----------------+-------------+
| Trial name                 | status   | loc            | optimizer   |
|----------------------------+----------+----------------+-------------|
| train_function_35c0f_00000 | RUNNING  | 172.28.0.2:374 | adam        |
| train_function_35c0f_00001 | PENDING  |                | adamW       |
| train_function_35c0f_00002 | PENDING  |                | SGD         |
| train_function_35c0f_00003 | PENDING  |                | adamW       |
| train_function_35c0f_0

Downloading:  15%|█▍        | 69.8M/478M [00:01<00:06, 68.1MB/s]
Downloading:  16%|█▌        | 76.3M/478M [00:01<00:06, 67.3MB/s]
Downloading:  17%|█▋        | 82.8M/478M [00:01<00:06, 67.5MB/s]
Downloading:  19%|█▊        | 89.4M/478M [00:01<00:06, 67.9MB/s]
Downloading:  20%|██        | 95.8M/478M [00:01<00:05, 67.9MB/s]
Downloading:  21%|██▏       | 102M/478M [00:01<00:05, 67.2MB/s] 
Downloading:  23%|██▎       | 109M/478M [00:01<00:05, 67.9MB/s]
Downloading:  24%|██▍       | 115M/478M [00:01<00:05, 67.7MB/s]
Downloading:  26%|██▌       | 122M/478M [00:01<00:05, 67.7MB/s]
Downloading:  27%|██▋       | 128M/478M [00:02<00:05, 67.7MB/s]
Downloading:  28%|██▊       | 135M/478M [00:02<00:05, 65.2MB/s]
Downloading:  30%|██▉       | 141M/478M [00:02<00:05, 65.5MB/s]
Downloading:  31%|███       | 148M/478M [00:02<00:05, 65.9MB/s]
Downloading:  32%|███▏      | 154M/478M [00:02<00:05, 66.0MB/s]
Downloading:  34%|███▎      | 160M/478M [00:02<00:05, 64.8MB/s]
Downloading:  35%|███▍      | 167M

...

== Status ==
Current time: 2022-04-14 09:09:50 (running for 00:17:23.81)
Memory usage on this node: 4.1/25.5 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: 0.7834224598930482 | Iter 4.000: 0.8061497326203209 | Iter 2.000: 0.7887700534759359 | Iter 1.000: 0.7513368983957219
Resources requested: 2.0/4 CPUs, 1.0/1 GPUs, 0.0/14.28 GiB heap, 0.0/7.14 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/train_function_2022-04-14_08-52-27
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+----------------------------+------------+-----------------+-------------+----------+------------+----------------------+
| Trial name                 | status     | loc             | optimizer   |     loss |   accuracy |   training_iteration |
|----------------------------+------------+-----------------+-------------+----------+------------+----------------------|
| train_function_35c0f_00009 | RUNNING    | 172.28.0.2:1434 | adam        |          |            |        

2022-04-14 09:11:01,651	INFO tune.py:639 -- Total run time: 1114.65 seconds (1114.27 seconds for the tuning loop).


Best trial config: {'num_epochs': 10, 'lr': 1e-05, 'batch_size': 16, 'weight_decay': 0.05, 'beta1': 0.9, 'beta2': 0.995, 'eps': 1e-08, 'max_gradient_norm': 8, 'd_h': 512, 'dropout': 0.1, 'factor': 0.5, 'patience': 2, 'optimizer': 'adam', 'momentum': 0.9}
Best trial final validation loss: 0.9082448921787242
Best trial final validation accuracy: 0.7941176470588235


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/

Best trial test set accuracy: 0.8212526389866291 , test loss : 0.8298243870916174
scheduler : dict_keys(['factor', 'min_lrs', 'patience', 'verbose', 'cooldown', 'cooldown_counter', 'mode', 'threshold', 'threshold_mode', 'best', 'num_bad_epochs', 'mode_worse', 'eps', 'last_epoch', '_last_lr'])
